# Interaciones de procesos

Mediante la instancia del Enviroment.process() podemos realizar la interración de procesos. Por ejemplo podemos tener un que esperar a que termine un proceso o interrumpir otro mientras se espera un evento.

## Espera de un proceso

Todo proceso en Simpy se puede ver como un evento. Por ejempo tenemos un vehiculo electrico que necesita recargar las baterias despues de un viaje.


In [25]:
import simpy
import random

In [6]:
DURACION_CARGA = 5
DURACION_BATERIA = 2

class Carro():
    def __init__(self, env):
        self.env = env
        #Empezamos el procesos cuando se instancia un nuevo objeto
        #Es importante definir la accion para realizar la interrupcion del evento.
        self.accion = env.process(self.run())
    def run(self):
        while True:
            print('Se parquea y empieza a regargar %d' % self.env.now)
            # Esperamos a que el proceso de recarga termine
            yield self.env.process(self.recarga())
            print('Se termino el proceso de recarga %d empieza el manejo' % self.env.now)
            yield self.env.timeout(DURACION_BATERIA)
            print('Termina el uso del carro manejo %d' % self.env.now)     
    def recarga(self):
        yield self.env.timeout(DURACION_CARGA)
    
            
            

In [10]:
env = simpy.Environment() # Creamos el ambiente de trabajo
carro1 = Carro(env)
env.run(until=35)

Se parquea y empieza a regargar 0
Se termino el proceso de recarga 5 empieza el manejo
Termina el uso del carro manejo 7
Se parquea y empieza a regargar 7
Se termino el proceso de recarga 12 empieza el manejo
Termina el uso del carro manejo 14
Se parquea y empieza a regargar 14
Se termino el proceso de recarga 19 empieza el manejo
Termina el uso del carro manejo 21
Se parquea y empieza a regargar 21
Se termino el proceso de recarga 26 empieza el manejo
Termina el uso del carro manejo 28
Se parquea y empieza a regargar 28
Se termino el proceso de recarga 33 empieza el manejo


# Interrumpir el proceso de ejecución
Ahora, imaginemos que no debemos esperar a que se recarge completamente la bateria, sino interrumpir el proceso de carga y simplemente empezar a conducir. Para ello simpy permite realizar este proceso a traves del método interrupt()

In [29]:
#Definimos el metodo que puede interrumpir la carga de la bateria
def manejar_ahora(env, carro):
    yield env.timeout(random.randint(1,20))
    carro.accion.interrupt()

In [30]:
DURACION_CARGA = 5
DURACION_BATERIA = 2

class CarroInterupcion():
    def __init__(self, env):
        self.env = env
        #Empezamos el procesos cuando se instancia un nuevo objeto
        #Es importante definir la accion para realizar la interrupcion del evento.
        self.accion = env.process(self.run())
    def run(self):
        while True:
            print('Se parquea y empieza a regargar %d' % self.env.now)
            # Esperamos a que el proceso de recarga termine
            try:
                yield self.env.process(self.recarga())
            except simpy.Interrupt:
                # Se ejecuta cuando recibe una interrupcion, se para la recarga
                # y pasamos al estado de manejar
                print('Se interrumpe carga %d'% self.env.now)
            print('Se termino el proceso de recarga %d empieza el manejo' % self.env.now)
            yield self.env.timeout(DURACION_BATERIA)   
    def recarga(self):
        yield self.env.timeout(DURACION_CARGA)

In [31]:
env = simpy.Environment()
carro1 = CarroInterupcion(env)
env.process(manejar_ahora(env, carro1))
env.run(until=25)

Se parquea y empieza a regargar 0
Se termino el proceso de recarga 5 empieza el manejo
Se parquea y empieza a regargar 7
Se interrumpe carga 11
Se termino el proceso de recarga 11 empieza el manejo
Se parquea y empieza a regargar 13
Se termino el proceso de recarga 18 empieza el manejo
Se parquea y empieza a regargar 20


# Ejemplo de interrupciones

Enunciado: Se tiene un numero de n maquinas trabajando continuamente dentro de una empresa de fabricacion de piezas de metal. Estas maquinas funcionan todo el día interrumpidamente. Cada maquina se descompone periodicamente al azar. 
Las reparaciones lo realiza un tecnico, este tecnico tiene otras tareas menos importantes que realiza dentro del empresa. Al momento de que una maquina falla automaticamente es notificado al tecnico y es prioridad, por lo que el tecnico deja de hacer su trabajo y empieza el arreglo de la maquina. Al finalizar el arreglo el tecnico continua con sus actividades diarias.

Una maquina tiene dos procesos, el trabajo de fabricar piezas y la interrupcion / daño que puede tener la maquina. Finalmente se tiene un solo tecnico dentro de la empresa.

In [32]:
# Definimos los parametros iniciales

PT_FABRICACION = 10 # TIEMPO PROMEDIO DE FABRICACION DE UNA PIEZA
TP_FALLA = 300 # TIEMPO EN EL QUE PUEDE FALLAR UNA MAQUINA
T_REPARACION = 30 # DURACION DE REPARAR UNA MAQUINA DEL TECNICO
T_OTROSTRABAJOS = 30 # DURACION DE OTROS TRABAJOS DEL TECNICO
N_MAQUINAS = 10 # NUMERO DE MAQUINAS QUE TIENE LA EMPRESA
SEMANAS = 4 # NUMERO DE SEMANAS DE LA SIMUACION
TIEMPO_SIMULACION = SEMANAS * 7 * 24 * 60 # TIEMPO DE SIMULACION DE LAS MAQUINAS 24/7

In [33]:
def tiempo_fabricacion():
    return random.normalvariate(PT_FABRICACION, 2)

def tiempo_falla():
    return random.expovariate(1/TP_FALLA)


In [43]:
class Maquina():
    def __init__(self, env, nombre, tecnico):
        self.env = env
        self.nombre = nombre
        self.numero_piezas = 0
        self.falla = False
        self.numero_fallas = 0
        self.proceso = env.process(self.trabajo(tecnico))
        env.process(self.falla_maquina())
    
    def trabajo(self, tecnico):
            while True:
                demora_pieza = tiempo_fabricacion()
                while demora_pieza:
                    try:
                        inicio = self.env.now
                        yield self.env.timeout(demora_pieza)
                        demora_pieza = 0
                    except simpy.Interrupt:
                        self.numero_fallas += 1
                        self.falla = True
                        demora_pieza -= self.env.now - inicio
                        with tecnico.request(priority=1) as requerimiento:
                            yield requerimiento
                            yield self.env.timeout(T_REPARACION)
                        self.falla = False
                self.numero_piezas += 1  
    
    def falla_maquina(self):
        while True:
            yield self.env.timeout(tiempo_falla())
            if not self.falla:
                self.proceso.interrupt()           

In [44]:
def trabajo_tecnico(env, tecnico):
    while True:
        tiempo = T_OTROSTRABAJOS
        while tiempo:
            with tecnico.request(priority=2) as requerimiento:
                yield requerimiento;
                try:
                    inicio = env.now
                    yield env.timeout(tiempo)
                    tiempo = 0
                except simpy.Interrupt:
                    tiempo -= env.now - inicio

In [45]:
print('Fabricación de piezas UPS')

env = simpy.Environment()
tecnico = simpy.PreemptiveResource(env, capacity=1)
maquinas = [Maquina(env, 'Maquina %d' % i, tecnico) for i in range (N_MAQUINAS)]
env.process(trabajo_tecnico(env, tecnico))
env.run(until=TIEMPO_SIMULACION)
print('Simulacion realizada despues de %d semanas' % SEMANAS)
for maquina in maquinas:
    print('Nombre: %s ha realizado %d partes con numero fallas %d' % (maquina.nombre, maquina.numero_piezas, maquina.numero_fallas))

Fabricación de piezas UPS
Simulacion realizada despues de 4 semanas
Nombre: Maquina 0 ha realizado 3341 partes con numero fallas 109
Nombre: Maquina 1 ha realizado 3342 partes con numero fallas 104
Nombre: Maquina 2 ha realizado 3324 partes con numero fallas 109
Nombre: Maquina 3 ha realizado 3359 partes con numero fallas 101
Nombre: Maquina 4 ha realizado 3318 partes con numero fallas 111
Nombre: Maquina 5 ha realizado 3277 partes con numero fallas 114
Nombre: Maquina 6 ha realizado 3290 partes con numero fallas 119
Nombre: Maquina 7 ha realizado 3373 partes con numero fallas 100
Nombre: Maquina 8 ha realizado 3350 partes con numero fallas 104
Nombre: Maquina 9 ha realizado 3303 partes con numero fallas 105


## Practica
En base a la practica del COVID vacunación, realizar el proceso de interrupción en base a que el 5% de los pacientes que estan en la sala de espera presentan complicaciones de salud y deben ser trasladados inmediamente al centro de salud más cercano y ser atendidos. Realizar este proceso y generar una segunda linea de simulación para el traslado, atencion y alta del paciente que presenta complicaciones, en base a los siguientes datos:
* Traslado de la ambulancia: 5 - 15 minutos.
* Atención centro de salud: 5 - 24 horas
* Alta: 1 - 2 horas
* El 10% de las personas que presentan complicaciones fallecen.

Generar graficas que indiquen las personas que presentaron complicaciones en base a los tiempos, estado de las personas y respuesta.




Referencias:
   * [1] https://interactivechaos.com/es/recursos-educativos/funciones-python?title=random
   * [2] https://interactivechaos.com/es/python/function/randomexpovariate
        